In [1]:
from __future__ import print_function

In [2]:
import torch
import numpy as np

# data

In [3]:
a = torch.tensor(np.random.randn(2,3),dtype=torch.float32)

In [4]:
a

tensor([[-0.1916,  1.3776, -0.5148],
        [ 0.2492, -0.6255,  0.5719]])

In [5]:
b = torch.tensor(np.random.randn(2,3),dtype=torch.float32)

In [6]:
b

tensor([[-1.1233, -0.3184, -0.3484],
        [ 1.2096, -1.6153,  0.2799]])

In [7]:
a*b

tensor([[ 0.2152, -0.4387,  0.1793],
        [ 0.3014,  1.0104,  0.1601]])

# 2 model

In [9]:
torch.nn.Embedding?

In [11]:
torch.cat?

In [12]:
class NET(torch.nn.Module):
    def __init__(self,num_users,num_iterms,num_embed,num_embed2,hidden = None):
        super(NET,self).__init__()
        self.num_users = num_users
        self.num_iterms = num_iterms
        
        self.user_embed = torch.nn.Embedding(self.num_users,num_embed)
        self.item_embed = torch.nn.Embedding(self.num_iterms,num_embed)
        
        self.user_embed2 = torch.nn.Embedding(self.num_users,num_embed2)
        self.item_embed2 = torch.nn.Embedding(self.num_iterms,num_embed2)
        
        #if pretrain set user_embed.weight = 
        self.mlp = torch.nn.ModuleList()
        in_dim = num_embed*2 + num_embed2
        for i in hidden:
            fc = torch.nn.Linear(in_dim,i)
            in_dim = i
            self.mlp.append(fc)
        
        ##last hidden
        self.h1 = torch.nn.Linear(hidden[-1],1)
        
    def forward(self,user,item):
        user_embed1 = self.user_embed(user)
        item_embed1 = self.item_embed(item)
        
        user_embed2 = self.user_embed2(user)
        item_embed2 = self.item_embed2(item)
        ##element product
        product_embed = user_embed2*item_embed2
        
        cat_embed = torch.cat((user_embed1,item_embed1,product_embed),dim=1)
        
        ##mlp
        in_vec = cat_embed
        for i in range(len(self.mlp)):
            out_vec = self.mlp[i](in_vec)
            out_vec = torch.nn.ReLU()(out_vec)
            in_vec = out_vec
        
        out = self.h1(out_vec)
        return torch.nn.Sigmoid()(out)

In [14]:
net = NET(4,4,3,2,hidden=[2,3,4])

In [20]:
user = torch.tensor([3,2],dtype=torch.long)

In [21]:
item = torch.tensor([1,2],dtype=torch.long)

In [22]:
out = net(user,item)

In [23]:
out

tensor([[0.5555],
        [0.5540]], grad_fn=<SigmoidBackward>)